In [ ]:
import os
import os
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from sklearn.model_selection import train_test_split

In [ ]:

# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Set dataset path
dataset_path = "/content/drive/MyDrive/dsgp datasets/"


Mounted at /content/drive


In [ ]:
# ✅ Check if the dataset exists
if not os.path.exists(dataset_path):
    print(f"❌ Error: Dataset path '{dataset_path}' not found. Please check your Google Drive folder.")
else:
    print(f"✅ Dataset found at: {dataset_path}")

✅ Dataset found at: /content/drive/MyDrive/dsgp datasets/


In [ ]:
import os
import cv2
import numpy as np
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.applications.resnet50 import preprocess_input

# ✅ Corrected folder mapping
plant_deficiency_mapping = {
    "Rice folder": ["Potassium(K)", "Nitrogen(N)", "Phosphorus(P)"],
    "Banana folder": ["IronB", "MagnesiumB", "PotassiumB"],
    "Coffee folder": ["PotassiumC", "IronC", "MagnesiumC"]
}

# ✅ Define the dataset path
dataset_path = "/content/drive/MyDrive/dsgp datasets"

# ✅ Function to preprocess images
def preprocess_image(image_path):
    try:
        img = cv2.imread(image_path)
        if img is None:
            return None
        img = cv2.resize(img, (224, 224))  # Resize for ResNet50
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  # Convert to RGB
        img = img_to_array(img)  # Convert to array
        img = preprocess_input(img)  # Preprocess for ResNet50
        return img
    except Exception as e:
        print(f"⚠️ Error processing {image_path}: {e}")
        return None

# ✅ Load images and labels
X_data, y_labels = [], []
label_mapping = {d: i for i, d in enumerate(sum(plant_deficiency_mapping.values(), []))}  # Map labels
missing_images = []  # Store paths of images that couldn't be read

# ✅ Process dataset
for plant, deficiencies in plant_deficiency_mapping.items():
    for deficiency in deficiencies:
        deficiency_path = os.path.join(dataset_path, plant, deficiency)
        if not os.path.exists(deficiency_path):
            print(f"⚠️ Warning: Directory '{deficiency_path}' not found, skipping...")
            continue

        # Process images
        for img_name in os.listdir(deficiency_path):
            img_path = os.path.join(deficiency_path, img_name)
            if not img_name.lower().endswith(('.png', '.jpg', '.jpeg')):  # Skip non-image files
                continue

            img = preprocess_image(img_path)
            if img is not None:
                X_data.append(img)
                y_labels.append(label_mapping[deficiency])
            else:
                missing_images.append(img_path)

# ✅ Convert to NumPy arrays
X_data, y_labels = np.array(X_data), np.array(y_labels)

print(f"✅ Loaded {len(X_data)} images with {len(label_mapping)} unique classes.")
if missing_images:
    print(f"⚠️ {len(missing_images)} images could not be read.")


✅ Loaded 1884 images with 9 unique classes.


In [ ]:
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Model
from tensorflow.keras.layers import GlobalAveragePooling2D

# ✅ Load the ResNet50 model (without the top layer)
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# ✅ Add a GlobalAveragePooling layer to flatten the features
x = base_model.output
x = GlobalAveragePooling2D()(x)

# ✅ Create a model for feature extraction
feature_extractor = Model(inputs=base_model.input, outputs=x)

# ✅ Define a function to extract features from an image
def extract_features(img):
    # Reshape and expand the dimensions of the image to match the input format of ResNet50
    img = np.expand_dims(img, axis=0)  # Add batch dimension
    # Extract features using ResNet50 model
    features = feature_extractor.predict(img)
    return features.flatten()  # Flatten the features to 1D

# ✅ Extract features for all images
X_features = []
for img in X_data:
    features = extract_features(img)
    X_features.append(features)

# ✅ Convert features to a NumPy array
X_features = np.array(X_features)

print(f"✅ Extracted features for {len(X_features)} images.")


1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 303ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 424ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 423ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 439ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 416ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 435ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 498ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 436ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 450ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 437ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 407ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 305ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 247ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 301ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 253ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 273ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 289ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 273ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 269ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 303ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 274ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 266ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 304ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 259ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 

In [ ]:
# ✅ Severity Mapping for each deficiency (You can modify this based on your dataset)
severity_mapping = {
    "Potassium(K)": "moderate",  # Example: moderate severity
    "Nitrogen(N)": "mild",  # Example: mild severity
    "Phosphorus(P)": "severe",  # Example: severe severity
    "IronB": "mild",
    "MagnesiumB": "moderate",
    "PotassiumB": "severe",
    "PotassiumC": "mild",
    "IronC": "severe",
    "MagnesiumC": "moderate"
}

# ✅ Encode severity labels as numeric values (e.g., mild=0, moderate=1, severe=2)
severity_encoded = {
    "mild": 0,
    "moderate": 1,
    "severe": 2
}

# ✅ Map severity labels to numeric values
y_severity = []
for deficiency in y_labels:
    plant_deficiency = list(label_mapping.keys())[list(label_mapping.values()).index(deficiency)]
    severity_label = severity_mapping.get(plant_deficiency, "mild")  # Default to "mild" if not found
    severity_numeric = severity_encoded.get(severity_label, 0)  # Default to 0 if not found
    y_severity.append(severity_numeric)

# ✅ Convert to NumPy array
y_severity = np.array(y_severity)

print(f"✅ Encoded severity labels: {len(set(y_severity))} unique severity levels.")


✅ Encoded severity labels: 3 unique severity levels.


In [ ]:
from sklearn.model_selection import train_test_split

# ✅ Split data into train and test sets (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(X_features, y_severity, test_size=0.2, random_state=42)

print(f"✅ Train data: {X_train.shape[0]} samples")
print(f"✅ Test data: {X_test.shape[0]} samples")


✅ Train data: 1507 samples
✅ Test data: 377 samples


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.callbacks import EarlyStopping

# ✅ Create a simpler neural network model with Batch Normalization
model = Sequential()

# Input layer (flatten the features)
model.add(Dense(256, input_dim=X_train.shape[1], activation='relu'))  # Reduced units
model.add(BatchNormalization())  # Batch Normalization

# Add a Dropout layer to prevent overfitting
model.add(Dropout(0.3))  # Reduced Dropout rate

# Hidden layer
model.add(Dense(128, activation='relu'))  # Reduced number of neurons
model.add(BatchNormalization())  # Batch Normalization

# Add another Dropout layer
model.add(Dropout(0.3))  # Reduced Dropout rate

# Output layer (3 classes: mild, moderate, severe)
model.add(Dense(3, activation='softmax'))

# ✅ Compile the model with Adam optimizer
model.compile(loss='sparse_categorical_crossentropy',
              optimizer=RMSprop(),
              metrics=['accuracy'])

# ✅ EarlyStopping and ReduceLROnPlateau callbacks with adjusted patience
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)  # Increased patience
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=0.0001)

# ✅ Train the model with accuracy tracking
history = model.fit(X_train, y_train,
                    validation_data=(X_test, y_test),
                    epochs=50,  # You can adjust the number of epochs as needed
                    batch_size=32,  # You can adjust the batch size as needed
                    callbacks=[early_stopping, reduce_lr])  # Add the callbacks here

# ✅ Evaluate the model on the test data
test_loss, test_accuracy = model.evaluate(X_test, y_test)

print(f"✅ Test Accuracy: {test_accuracy:.4f}")


Epoch 1/50
48/48 ━━━━━━━━━━━━━━━━━━━━ 2s 17ms/step - accuracy: 0.6372 - loss: 0.9579 - val_accuracy: 0.7162 - val_loss: 0.6909 - learning_rate: 0.0010
Epoch 2/50
48/48 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.8205 - loss: 0.4567 - val_accuracy: 0.8090 - val_loss: 0.4832 - learning_rate: 0.0010
Epoch 3/50
48/48 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.8763 - loss: 0.3139 - val_accuracy: 0.7825 - val_loss: 0.5522 - learning_rate: 0.0010
Epoch 4/50
48/48 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - accuracy: 0.8901 - loss: 0.2644 - val_accuracy: 0.8541 - val_loss: 0.3662 - learning_rate: 0.0010
Epoch 5/50
48/48 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step - accuracy: 0.9160 - loss: 0.2325 - val_accuracy: 0.8515 - val_loss: 0.3540 - learning_rate: 0.0010
Epoch 6/50
48/48 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step - accuracy: 0.9291 - loss: 0.1869 - val_accuracy: 0.8488 - val_loss: 0.3414 - learning_rate: 0.0010
Epoch 7/50
48/48 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step - accuracy: 0.9249 - loss: 0.1855 - val_accur

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.callbacks import EarlyStopping

# ✅ Create the neural network model
model = Sequential()

# Input layer (flatten the features)
model.add(Dense(512, input_dim=X_train.shape[1], activation='relu'))

# Add a Dropout layer to prevent overfitting
model.add(Dropout(0.5))

# Hidden layer
model.add(Dense(256, activation='relu'))

# Add another Dropout layer
model.add(Dropout(0.5))

# Output layer (3 classes: mild, moderate, severe)
model.add(Dense(3, activation='softmax'))

# ✅ Compile the model with RMSprop optimizer
model.compile(loss='sparse_categorical_crossentropy',
              optimizer=RMSprop(),
              metrics=['accuracy'])

from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

# ✅ EarlyStopping and ReduceLROnPlateau callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=0.0001)

# ✅ Train the model with accuracy tracking
history = model.fit(X_train, y_train,
                    validation_data=(X_test, y_test),
                    epochs=50,  # You can adjust the number of epochs as needed
                    batch_size=32,  # You can adjust the batch size as needed
                    callbacks=[early_stopping, reduce_lr])  # Add the callbacks here

# ✅ Evaluate the model on the test data
test_loss, test_accuracy = model.evaluate(X_test, y_test)

print(f"✅ Test Accuracy: {test_accuracy:.4f}")


Epoch 1/50
48/48 ━━━━━━━━━━━━━━━━━━━━ 3s 35ms/step - accuracy: 0.4544 - loss: 2.1807 - val_accuracy: 0.6446 - val_loss: 0.7727 - learning_rate: 0.0010
Epoch 2/50
48/48 ━━━━━━━━━━━━━━━━━━━━ 2s 18ms/step - accuracy: 0.6866 - loss: 0.7304 - val_accuracy: 0.6605 - val_loss: 0.7660 - learning_rate: 0.0010
Epoch 3/50
48/48 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step - accuracy: 0.7237 - loss: 0.6484 - val_accuracy: 0.8621 - val_loss: 0.3437 - learning_rate: 0.0010
Epoch 4/50
48/48 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step - accuracy: 0.8132 - loss: 0.4767 - val_accuracy: 0.7745 - val_loss: 0.5710 - learning_rate: 0.0010
Epoch 5/50
48/48 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step - accuracy: 0.8272 - loss: 0.4483 - val_accuracy: 0.8064 - val_loss: 0.3864 - learning_rate: 0.0010
Epoch 6/50
48/48 ━━━━━━━━━━━━━━━━━━━━ 1s 20ms/step - accuracy: 0.8389 - loss: 0.3948 - val_accuracy: 0.8753 - val_loss: 0.3070 - learning_rate: 0.0010
Epoch 7/50
48/48 ━━━━━━━━━━━━━━━━━━━━ 1s 24ms/step - accuracy: 0.8669 - loss: 0.3726 - val_acc

In [ ]:
# Retrieve training accuracy from history
train_accuracy = history.history['accuracy']

# Print the last training accuracy (after the final epoch)
print(f"✅ Final Training Accuracy: {train_accuracy[-1]:.4f}")


✅ Final Training Accuracy: 0.9914
